In [ ]:
import pickle
from os import path

import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.backend import mean, square

from spektral.datasets import qm9
from spektral.layers import EdgeConditionedConv, GlobalSumPool, GlobalAttentionPool
from spektral.utils import label_to_one_hot

from sklearn.preprocessing import StandardScaler, PowerTransformer

In [ ]:
learning_rate = 1e-3
epochs = 10
batch_size = 32

In [ ]:
A_all, X_all, E_all, y_all = qm9.load_data(return_type='numpy',
                           nf_keys='atomic_num',
                           ef_keys='type',
                           self_loops=True,
                           amount=None) # load entire dataset
# Preprocessing
X_uniq = np.unique(X_all)
X_uniq = X_uniq[X_uniq != 0]
E_uniq = np.unique(E_all)
E_uniq = E_uniq[E_uniq != 0]

X_all = label_to_one_hot(X_all, X_uniq)
E_all = label_to_one_hot(E_all, E_uniq)

In [ ]:
# Parameters
N = X_all.shape[-2]       # Number of nodes in the graphs
F = X_all[0].shape[-1]    # Dimension of node features
S = E_all[0].shape[-1]    # Dimension of edge features
n_out = y_all.shape[-1]   # Dimension of the target

In [ ]:
# because we don't want to train only on the lightest molecules
# we randomly sample from the dataset
indices = np.random.choice(X_all.shape[0], 10000, replace=False)
X = X_all[indices, :, :]
A = A_all[indices, :, :]
E = E_all[indices, :, :, :]
y = y_all.iloc[indices, :].copy()

In [ ]:
# Applying Yeo-Johnson transformation
# to reduce skew and standardize data
task_to_scaler = dict()
for task in list(y.columns)[1:]:
    # scaler = StandardScaler()
    scaler = PowerTransformer()
    y.loc[:, task] = scaler.fit_transform(y[[task]])
    task_to_scaler[task] = scaler

In [ ]:
clusters = [['A', 'B', 'alpha'], 
            ['C', 'r2', 'u0'],
            ['zpve', 'g298', 'cv'],
            ['lumo', 'u298', 'h298'],
            ['mu', 'homo']]

In [ ]:
A_train, A_test, \
    X_train, X_test, \
    E_train, E_test, \
    y_train, y_test = train_test_split(A, X, E, y, test_size=0.1)

In [ ]:
def build_single_task_model(*, N, F, S):
    X_in = Input(shape=(N, F))
    A_in = Input(shape=(N, N))
    E_in = Input(shape=(N, N, S))

    gc1 = EdgeConditionedConv(64, activation='relu')([X_in, A_in, E_in])
    gc2 = EdgeConditionedConv(128, activation='relu')([gc1, A_in, E_in])
    pool = GlobalAttentionPool(256)(gc2)
    dense = Dense(256, activation='relu')(pool)
    output = Dense(1)(dense)

    # Build model
    model = Model(inputs=[X_in, A_in, E_in], outputs=output)
    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')

    return model

In [ ]:
def build_hard_sharing_model(*, N, F, S, num_tasks):
    X_in = Input(shape=(N, F))
    A_in = Input(shape=(N, N))
    E_in = Input(shape=(N, N, S))

    gc1 = EdgeConditionedConv(64, activation='relu')([X_in, A_in, E_in])
    gc2 = EdgeConditionedConv(128, activation='relu')([gc1, A_in, E_in])
    pool = GlobalAttentionPool(256)(gc2)
    dense_list = [Dense(256, activation='relu')(pool) for i in range(num_tasks)]
    output_list = [Dense(1)(dense_layer) for dense_layer in dense_list]

    # Build model
    model = Model(inputs=[X_in, A_in, E_in], outputs=output_list)
    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')

    return model

In [ ]:
def build_soft_sharing_model(*, N, F, S, num_tasks, share_param):
    X_in = Input(shape=(N, F))
    A_in = Input(shape=(N, N))
    E_in = Input(shape=(N, N, S))

    gc1_list = [EdgeConditionedConv(64, activation='relu')([X_in, A_in, E_in]) for i in range(num_tasks)]
    gc2_list = [EdgeConditionedConv(128, activation='relu')([gc1, A_in, E_in]) for gc1 in gc1_list]
    pool_list = [GlobalAttentionPool(256)(gc2) for gc2 in gc2_list]
    dense_list = [Dense(256, activation='relu')(pool) for pool in pool_list]
    output_list = [Dense(1)(dense) for dense in dense_list]

    def loss(y_actual, y_pred):
        avg_layer_diff = 0
        for i in range(num_tasks):
            for j in range(i):
                for gc in [gc1_list, gc2_list]:
                      avg_layer_diff += mean(square(gc[i].trainable_weights - gc[j].trainable_weights))
        avg_layer_diff /= (num_tasks)*(num_tasks-1)/2  
        return mean(square(y_actual - y_pred)) + share_param*avg_layer_diff

    # Build model
    model = Model(inputs=[X_in, A_in, E_in], outputs=output_list)
    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')

    return model

In [ ]:
# FOLDER_PATH = '/content/drive/My Drive/Colab Notebooks/demo_models'
FOLDER_PATH = 'demo_models'

def generate_model_filename(tasks):
    filename = "".join(sorted(tasks))
    return path.join(FOLDER_PATH, f'{filename}.h5')

def generate_task_scaler_filename(task):
    return path.join(FOLDER_PATH, f'{task}_scaler.pkl')

In [ ]:
def save_model(model, tasks):
    model.save_weights(generate_model_filename(tasks))
    for task in tasks:
        scaler_filename = generate_task_scaler_filename(task)
        with open(scaler_filename, 'wb') as f:
#       print(task_to_scaler[task].mean_[0], file=f)
#       print(task_to_scaler[task].scale_[0], file=f)
            scaler = task_to_scaler[task]
            pickle.dump(obj=scaler, file=f)

def load_hard_sharing_model(*, N, F, S, tasks, task_to_scaler=dict()):
    model = build_hard_sharing_model(N=N, F=F, S=S, num_tasks=len(tasks))
    model.load_weights(generate_model_filename(tasks))
    for task in tasks:
        if task not in task_to_scaler:
            with open(generate_task_scaler_filename(task), 'rb') as f:
                task_to_scaler[task] = pickle.load(f)
    return model, task_to_scaler

In [ ]:
def predict_property(prop, mol_id, clusters, N=N, F=F, S=S):
    for cluster in clusters:
        if prop in cluster:
            model, task_to_scaler = load_hard_sharing_model(N=N, F=F, S=S, tasks=cluster, task_to_scaler=task_to_scaler)
            i = mol_id - 1

            # convert shape for batch mode
            def wrap(a):
                return a.reshape([1] + list(a.shape))
            x = list(map(wrap, [X_all[i], A_all[i], E_all[i]]))
            
            cluster_prediction = model.predict(x)
            prediction = cluster_prediction[cluster.index(prop)]
            prediction = task_to_scaler[prop].inverse_transform(prediction)
            return prediction[0][0]

In [ ]:
if __name__ == '__main__' and '__file__' not in globals():
    print('begin training models')
    for cluster in clusters:
        print(f'training {cluster}')
        model = build_hard_sharing_model(N=N, F=F, S=S, num_tasks=len(cluster))
        y_train_cluster = np.hsplit(y_train[cluster].values, len(cluster))
        model.fit(x=[X_train, A_train, E_train], 
                  y=y_train_cluster,
                  batch_size=128,
                  validation_split=0.1,
                  epochs=3)
        save_model(model, cluster)

In [ ]:
if __name__ == '__main__' and '__file__' not in globals():   
    for cluster in clusters:
        model, _ = load_hard_sharing_model(N=N, F=F, S=S, tasks=clusters[0])
        y_test_cluster = np.hsplit(y_test[cluster].values, len(cluster))
        model_loss = model.evaluate(x=[X_test, A_test, E_test],
                                    y=y_test_cluster)
        print(f"Test loss: {model_loss}")

In [ ]:
if __name__ == '__main__' and '__file__' not in globals():
    print(predict_property('A', 1, clusters, N=N, F=F, S=S))